In [1]:
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import plot, init_notebook_mode
from sklearn.preprocessing import LabelEncoder
import numpy as np
from hdbscan import HDBSCAN

In [2]:
data_1D = pd.read_csv('/home/eguimard/PycharmProjects/DataStudy/DataStorage/Results/DataAnalyse/CPUWarningCriticalVM2/cpu/data/reduced_1D_data.csv')

In [3]:
data_1D['0']

0      34.737118
1      35.666020
2      13.312069
3      15.760091
4      16.723112
         ...    
115    35.690693
116    15.074228
117    35.221570
118    14.708056
119    34.983950
Name: 0, Length: 120, dtype: float64

In [4]:
data_2D = pd.read_csv('/home/eguimard/PycharmProjects/DataStudy/DataStorage/Results/DataAnalyse/CPUWarningCriticalVM2/cpu/data/reduced_2D_data.csv')

In [5]:
data_2D

,0,1
0,26.473200,11.040215
1,26.563128,10.795956
2,15.145804,5.717798
3,14.871727,7.014143
4,14.204561,7.300304
...,...,...
115,27.016798,10.913266
116,15.795841,7.202504
117,27.149965,10.789955
118,15.279580,6.447621


In [6]:
df_info = pd.read_csv('/home/eguimard/PycharmProjects/DataStudy/DataStorage/Results/DataAnalyse/CPUWarningCriticalVM2/cpu/data/saved_data.csv')

In [7]:
df_info

,utctimestamp,severity
0,2022-06-16 12:20:10.993737+00:00,INFO
1,2022-06-16 12:20:25.990961+00:00,INFO
2,2022-06-16 12:20:41.021435+00:00,INFO
3,2022-06-16 12:20:55.988691+00:00,INFO
4,2022-06-16 12:21:10.992139+00:00,INFO
...,...,...
115,2022-06-16 12:48:55.942189+00:00,INFO
116,2022-06-16 12:49:10.936065+00:00,INFO
117,2022-06-16 12:49:25.928817+00:00,INFO
118,2022-06-16 12:49:40.925928+00:00,INFO


In [8]:
timestamp_series = df_info['utctimestamp']
series_color = df_info['severity']

In [9]:
series_color = pd.Series(HDBSCAN().fit_predict(data_2D))
series_color[0] = -1

In [32]:
class Subplot:
    

    def __init__(self,list_discrete_color = None):

        if list_discrete_color is None:
            self.list_discrete_color = ['green', 'orange', 'red', 'blue', 'yellow', 'black', 'brown','pink','violet']
        else:
            self.list_discrete_color = list_discrete_color
    
    def add_scatter_2D_to_figure(self, figure, row, col,
                          series_x, series_y, color=None, name_and_group=None, mode='lines+markers',show_legend=True ):

        return figure.add_scatter(x=series_x, y=series_y, mode=mode, marker_size=10,
                              name=name_and_group, legendgroup=name_and_group,
                              showlegend=show_legend,col=col, row=row, marker_color=color)



    def create_scatter_plot_2D(self,  series_x, series_y, series_color=None, name_scatter='',  mode='lines+markers',
                               show_legend=True, figure=None, row=None, col=None):

        if None in [figure, row, col]:
            row, col = 1, 1
            figure = make_subplots(row, col,  subplot_titles=('reduced 1D data with time in x-axis',
                                                               'reduced 2D data'))

        if series_color is None:
            figure.add_scatter(x=series_x, y=series_y, mode=mode, line_color=self.list_discrete_color[3],
                               name=name_scatter, col=col, row=row, showlegend=show_legend)
        else:

            if mode != 'markers':
                figure.add_scatter(x=series_x, y=series_y, mode='lines', line_color='grey',
                                   name='state change', col=col, row=row)

            list_labels = list(dict(series_color.value_counts()).keys())

            for index in range(len(list_labels)):
                figure = self.add_scatter_2D_to_figure(figure, row, col,
                                                       series_x.where(series_color == list_labels[index]),
                                                       series_y.where(series_color == list_labels[index]),

                                                       color=self.list_discrete_color[index],
                                                       name_and_group=list_labels[index],
                                                       mode=mode,
                                                       show_legend=show_legend)
            
        #figure.update_layout(title=name_scatter)
        
        return figure

In [37]:
sub_fig = make_subplots(1,2, horizontal_spacing=0.1,subplot_titles=('reduced 1D data with time in x-axis',
                                                               'reduced 2D data'))

sub_fig = Subplot().create_scatter_plot_2D(timestamp_series, data_1D['0'], series_color,'','lines+markers',True,sub_fig,1,2)
sub_fig = Subplot().create_scatter_plot_2D( data_2D['0'], data_2D['1'], series_color, '', 'markers',False,sub_fig,1,1)



##


In [38]:
plot(sub_fig, filename='test.html', auto_open=True)

'test.html'